In [ ]:
import os

from tqdm.notebook import tqdm
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.colors as mpl_c
import matplotlib.patches as mpl_p

from py_bacy.intf_pytassim import clm, io
from pytassim.localization import GaspariCohn
from pytassim.assimilation.base import BaseAssimilation

In [ ]:
plt.style.use('paper')
plt.style.use('egu_journals')

In [ ]:
base_path = '/work/um0203/u300636/for2131/runs/'

In [ ]:
clm_const = clm.load_constant_data(os.path.join(base_path, 'utilities'))
clm_grid_xr = clm_const.isel(levels=[9]).stack(grid_points=['lat', 'lon', 'levels']).grid_points
clm_grid = BaseAssimilation()._grid_index_to_array(clm_grid_xr)

In [ ]:
ds_obs = io.load_observations(os.path.join(base_path, 'obs', 'ens', 't2m_obs_016_0_1_long.nc'))
obs_grid = ds_obs['obs_grid_1']
obs_grid = BaseAssimilation()._grid_index_to_array(obs_grid)

In [ ]:
gc = GaspariCohn((15000, 0.7), clm.distance_func)

In [ ]:
obs_equivalent = []
for gp in tqdm(clm_grid):
    tmp_equi = gc.localize_obs(gp, obs_grid)[1].sum()
    obs_equivalent.append(tmp_equi)
    
obs_equivalent = xr.DataArray(
    obs_equivalent,
    coords={'grid_points': clm_grid_xr},
    dims=['grid_points']
)
obs_equivalent = obs_equivalent.unstack('grid_points')

In [ ]:
figsize = [s*0.48 for s in plt.rcParams['figure.figsize']]
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
norm = mpl_c.Normalize(vmin=0.,vmax=2)
cf = ax.contourf(obs_equivalent['lon'].values, obs_equivalent['lat'].values, obs_equivalent.values.squeeze(),
                 levels=list(np.linspace(0, 2, 21))+[2.1], cmap='RdBu_r', norm=norm)

cbar = fig.colorbar(cf, fraction=0.046, pad=0.04)
cbar.set_ticks(np.arange(0, 2.5, 0.5))
cbar.set_label('Observational equivalent')
ax.set_xlim(7.491, 10.72)
ax.set_xticks(np.arange(7.5, 11, 0.5))
ax.set_xlabel(r'Longitude ($^\circ$)')
ax.set_ylabel(r'Latitude ($^\circ$)')
ax.spines['right'].set_visible(True)
ax.spines['top'].set_visible(True)
fig.savefig('../figures/fig_02_obs_equivalent.png', dpi=300)
plt.show()